In [ ]:
from utils_io import read_json, load_items

params = read_json('parameters.json')
RESIZE_X = params['resize']['x']
RESIZE_Y = params['resize']['y']
ITEM_FOLDER = params['item_folder']
items = load_items(ITEM_FOLDER)

hl = [] # histogram list

In [ ]:
from utils_sift import read_features_from_file, unpack_keypoint

views = ['top_01','top-side_01','top-side_02','bottom_01','bottom-side_01','bottom-side_02']
views = ['top_01']
for item in items:
    for view in views:
        try:
            filename = ITEM_FOLDER + '/' + item + '/' + item + '_' + view + '_sift.npy'
            kp, des = read_features_from_file(filename)
            kp, des = unpack_keypoint(kp, des)
            des = des.astype('float32')
            hl.append( (item, view, kp, des) )
        except IOError:
            pass
        except OSError:
            pass

In [ ]:
import numpy as np

ivdm = np.zeros( (len(hl),len(hl)) ) # Item/view distance matrix

In [ ]:
from utils_sift import calc_matches

for i, (it1, vi1, kp1, des1) in enumerate(hl):
    for j, (it2, vi2, kp2, des2) in enumerate(hl):
        if j>i:
            good = calc_matches(des1, des2)
            #ivdm[i][j] = float(len(good)) / float(len(des1))
            ivdm[i][j] = len(good)
            ivdm[j][i] = ivdm[i][j]

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
plt.imshow(ivdm,cmap='jet');

In [ ]:
n = len(items)
idm = np.zeros( (n, n) ) # item distance matrix

In [ ]:
n = len(hl)
for i in range(n):
    for j in range(i+1, n):
        iti = items.index(hl[i][0])
        itj = items.index(hl[j][0])
        if ivdm[i][j] > idm[iti][itj]:
            idm[iti][itj] = ivdm[i][j]
            idm[itj][iti] = idm[iti][itj]

In [ ]:
plt.imshow(idm,cmap='jet');

In [ ]:
def plot_distance(item, d):
    idx = items.index(item)
    plt.plot(idm[idx],'b-'); plt.plot(idm[idx],'bo'); plt.title(items[idx]); plt.plot([0,40],[d,d],'r-'); plt.show();
    for d, it in sorted([(dist, it) for it, dist in zip(items, idm[idx]) if dist > d], reverse=True):
        print('%d %s' % (d, it))

In [ ]:
from ipywidgets import interact
interact(plot_distance, item=items, d=(0,50,5));

In [ ]:
np.max(idm)

In [ ]:
plt.hist(idm.flatten());